In [ ]:
# 🎯 MODEL REGISTRATION SCRIPT (WITH SLACK NOTIFICATIONS ADDED)

import mlflow
from mlflow.tracking import MlflowClient
import sys
import yaml
import json
import traceback
import requests
from typing import Dict, Optional, List
from datetime import datetime
from pyspark.sql import SparkSession
import pandas as pd
from IPython import get_ipython

print("=" * 80)
print("🎯 MODEL REGISTRATION SYSTEM (CONTROLLED MULTI-RUN + SLACK NOTIFICATIONS)")
print("=" * 80)

# ---------------------- LOAD PIPELINE CONFIG ----------------------

try:
    with open("pipeline_config.yml", "r") as f:
        pipeline_cfg = yaml.safe_load(f)
    print("✅ pipeline_config.yml loaded")
except Exception as e:
    print(f"❌ Failed to load config: {e}")
    sys.exit(1)


class Config:
    def __init__(self):
        MODEL_TYPE = pipeline_cfg["model"]["type"]
        UC_CATALOG = pipeline_cfg["model"]["catalog"]
        UC_SCHEMA = pipeline_cfg["model"]["schema"]
        BASE_NAME = pipeline_cfg["model"]["base_name"]

        self.MODEL_NAME = f"{UC_CATALOG}.{UC_SCHEMA}.{BASE_NAME}_{MODEL_TYPE}"
        self.EXPERIMENT_NAME = pipeline_cfg["experiment"]["name"]
        self.ARTIFACT_PATH = pipeline_cfg["experiment"]["artifact_path"]

        self.PRIMARY_METRIC = pipeline_cfg["metrics"]["classification"]["primary_metric"]
        
        # Duplicate check settings
        self.TOLERANCE = pipeline_cfg["registry"]["duplicate_detection"]["tolerance"]
        self.METRICS_TO_COMPARE = pipeline_cfg["registry"]["duplicate_detection"]["metrics_to_compare"]
        self.DUPLICATE_CHECK_ENABLED = pipeline_cfg["registry"]["duplicate_detection"]["enabled"]

        self.EVALUATION_LOG_TABLE = pipeline_cfg["tables"]["evaluation_log"]

        # ---------------------- NEW: SLACK CONFIG ----------------------
        # Try shared-scope then dev-scope (same behavior as V2)
        self.SLACK_WEBHOOK = None
        for scope in ["shared-scope", "dev-scope"]:
            try:
                self.SLACK_WEBHOOK = dbutils.secrets.get(scope, "SLACK_WEBHOOK_URL")
                print(f"🔐 Slack webhook loaded from secret scope: {scope}")
                break
            except:
                pass

        if not self.SLACK_WEBHOOK:
            print("⚠ Slack webhook NOT configured. Notifications disabled.")


config = Config()

print(f"\n📌 Model Registry: {config.MODEL_NAME}")
print(f"📌 Duplicate Logic: {'ENABLED' if config.DUPLICATE_CHECK_ENABLED else 'DISABLED'}")
print(f"📌 Slack Enabled: {'YES' if config.SLACK_WEBHOOK else 'NO'}")
print("=" * 80)


# ---------------------- SLACK NOTIFIER (FROM V2) ----------------------

class SlackNotifier:
    def __init__(self, webhook_url: Optional[str]):
        self.webhook_url = webhook_url

    def send(self, message: str, level: str = "info"):
        if not self.webhook_url:
            print(f"📢 [Slack Disabled] {message}")
            return
        
        emoji = {"info":"ℹ️","success":"✅","warning":"⚠️","error":"❌"}.get(level,"ℹ️")
        payload = {"text": f"{emoji} {message}"}

        try:
            requests.post(self.webhook_url, json=payload, timeout=5)
            print(f"📨 Slack message sent: {level}")
        except Exception as e:
            print(f"⚠ Slack send failed: {e}")

slack = SlackNotifier(config.SLACK_WEBHOOK)


# ---------------------- INIT MLFLOW + SPARK ----------------------

spark = SparkSession.builder.appName("ModelRegistration").getOrCreate()
mlflow.set_tracking_uri("databricks")
mlflow.set_registry_uri("databricks-uc")
client = MlflowClient()

experiment = mlflow.get_experiment_by_name(config.EXPERIMENT_NAME)
if experiment is None:
    raise Exception(f"Experiment '{config.EXPERIMENT_NAME}' not found!")


# ---------------------- FETCH ALL RUNS ----------------------

def get_all_runs() -> List[Dict]:
    print("\n📍 Fetching ALL experiment runs...")

    runs = client.search_runs(
        [experiment.experiment_id],
        order_by=[f"metrics.{config.PRIMARY_METRIC} DESC"],
        max_results=500
    )

    return [
        {
            "run_id": run.info.run_id,
            "run_name": run.info.run_name or "unnamed_run",
            "metrics": {m: run.data.metrics.get(m) for m in config.METRICS_TO_COMPARE},
            "params": run.data.params,
            "primary_metric": run.data.metrics.get(config.PRIMARY_METRIC),
            "model_uri": f"runs:/{run.info.run_id}/{config.ARTIFACT_PATH}"
        }
        for run in runs
    ]


# ---------------------- DUPLICATE DETECTION ----------------------

def is_duplicate_model(new_model: Dict) -> bool:
    if not config.DUPLICATE_CHECK_ENABLED:
        return False

    versions = client.search_model_versions(f"name='{config.MODEL_NAME}'")

    for version in versions:
        run = client.get_run(version.run_id)

        metric_match = all(
            abs(run.data.metrics.get(m) - new_model["metrics"].get(m)) <= config.TOLERANCE
            for m in config.METRICS_TO_COMPARE
            if run.data.metrics.get(m) is not None and new_model["metrics"].get(m) is not None
        )

        params_match = (new_model["params"] == run.data.params)

        if metric_match and params_match:
            print(f"⚠ Duplicate detected → Matches version v{version.version}")
            slack.send(f"⚠ Duplicate model skipped (Run: {new_model['run_name']})", "warning")
            return True
    
    return False


# ---------------------- REGISTER MODEL ----------------------

def register_model(model: Dict):
    if is_duplicate_model(model):
        return None

    new_version = mlflow.register_model(model["model_uri"], config.MODEL_NAME)
    print(f"🎯 Registered version: {new_version.version}")

    slack.send(f"✅ Model registered: {config.MODEL_NAME} (v{new_version.version})", "success")

    return new_version.version


# ---------------------- LOG DECISION ----------------------

def log_decision(model, registered, version, reason):
    df = pd.DataFrame([{
        "timestamp": datetime.now(),
        "run_id": model["run_id"],
        "run_name": model["run_name"],
        "primary_metric_value": model["primary_metric"],
        "metrics_json": json.dumps(model["metrics"]),
        "params_json": json.dumps(model["params"]),
        "registered": registered,
        "registered_version": version,
        "reason": reason
    }])

    spark.createDataFrame(df).write.format("delta").mode("append").saveAsTable(config.EVALUATION_LOG_TABLE)
    print("📄 Logged in evaluation table")


# ---------------------- MAIN EXECUTION ----------------------

def main():
    print("\n🚀 Scanning all model runs...")
    runs = get_all_runs()

    for idx, model in enumerate(runs, start=1):
        print(f"\n---------- [{idx}/{len(runs)}] Processing: {model['run_name']} ----------")

        version = register_model(model)

        if version:
            log_decision(model, True, version, "✔ Registered (unique)")
        else:
            log_decision(model, False, None, "⚠ Duplicate - Skipped")

    slack.send("📁 Model registration pipeline completed.", "info")
    print("\n🎉 Registration Complete — All Runs Processed")


if __name__ == "__main__":
    main()
